<h1>Main Activity</h1>
<h2>setup</h2>

In [ ]:
#imports
import numpy as np
from scipy.io import loadmat

In [ ]:
#for better view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(edgeitems=30, linewidth=300,precision=2)

In [ ]:
#load data
Xtrue = loadmat("incomplete.mat")["Xtrue"]
Y1 = loadmat("incomplete.mat")["Y1"]
Y2 = loadmat("incomplete.mat")["Y2"]
Y3 = loadmat("incomplete.mat")["Y3"]

<h2>Iterative Singular Value Thresholding</h2>
<h3>From Period 15 Activity</h3>

In [ ]:
### DO NOT change
def ItSingValThresh(Y, r):
    """
    Iterative Singular Value Thresholding function for Matrix Completion
    Assumes elements of Y are integers between 0 and 10^6
    """
    tol = 10**(-6)  # difference between iterates at termination
    n,p = Y.shape 
    X = np.array(Y) #make a copy so operations do not mutate the original
    X[np.isnan(X)] = 0 # Fill in missing entries with zeros

    err = 10**6 
    
    while err > tol:
        U,s,VT = np.linalg.svd(X, full_matrices=False)
        V = VT.T ; S = np.diag(s)
        Xnew = (U[:,:r] @ S[:r,:r] @ V[:,:r].T).round()
        for i in range(n):
            for j in range(p):
                if Y[i,j] < 10**6 and Y[i,j] > 0:
                    Xnew[i,j] = Y[i,j]
        err = np.linalg.norm(X-Xnew,'fro') 
        X = Xnew
    return X

In [ ]:
Xtrue

In [ ]:
np.linalg.norm(Xtrue-ItSingValThresh(Y,2),'fro')**2 for Y in [Y1]

In [ ]:
np.linalg.norm(Xtrue-ItSingValThresh(Y2,2),'fro')**2

In [ ]:
np.linalg.norm(Xtrue-ItSingValThresh(Y3,2),'fro')**2

<h2>Soft Impute</h2>
<h3>for part a)</h3>

In [ ]:
### DO NOT change
def SoftImpute(X, params, r):
    '''
    soft impute algorithm for one lambda
    code based on lecture and paper: https://arxiv.org/pdf/1401.2451.pdf (Algorithm 1)
    :param np.ndarray X: the matrix to be completed
    :param np.array params: strictly decreasing regularization parameters [lambda_1 ... lambda_k]
    :param int r: expected rank of the complete matrix
    :return the completed matrix
    '''
    Z=np.zeros_like(X)
    n,p=X.shape
    ans=[]
    for lam in params:
        eps = 10**(-1) 
        y = eps + 1
        while y > eps:
            for i in range(n):
                for j in range(p):
                    if X[i,j] < 10**6 and X[i,j] > 0:
                        Z[i,j] = X[i,j]
            U,s,VT = np.linalg.svd(Z)
            if(max(s)<lam):
                break
            slam=[max(0,i-lam) for i in s]; V=VT.T; S = np.diag(slam)
            Znew = (U[:,:r] @ S[:r,:r] @ V[:,:r].T)
            y = np.linalg.norm(Znew-Z,'fro')**2 / np.linalg.norm(Z,'fro')**2
            Z = Znew
        ans.append(Z)
    return ans

In [ ]:
#students are to come up with arguments for the Soft Impute algorithm
#for Y1, try something close to 1 to a list of integer power
#try rank 2 and 3
#params,r=
[np.linalg.norm(Xtrue-Z,'fro')**2 for Z in SoftImpute(Y1,params,2)]

In [ ]:
#students are to come up with arguments for the Soft Impute algorithm
#for Y2 and Y3, try an array that's less than a hundred that decrease by half each time
#try rank 2 and 3
#params,r=
[np.linalg.norm(Xtrue-Z,'fro')**2 for Z in SoftImpute(Y2,params,r)]
[np.linalg.norm(Xtrue-Z,'fro')**2 for Z in SoftImpute(Y3,params,r)]

<h2>Randomized SVD</h2>
<h3>for part b)</h3>

In [ ]:
### DO NOT change
def RandSVD(B,k,p,q):
    '''
    randomized SVD for matrix completion
    from paper:https://arxiv.org/pdf/1401.2451.pdf (Algorithm 2)
    :param np.ndarray B: the matrix to be completed
    :param int k: expected rank of the complete matrix
    :param p: oversampling projection vectors
    :param q: exponent
    :return the completed matrix
    '''
    
    A=B.copy()
    m,n=A.shape
    A[np.isnan(A)]=0
    O=np.random.normal(size=(n,k+p))
    Y=np.linalg.matrix_power(A@A.T,q)@A@O
    V,R=np.linalg.qr(Y)
    B=(V.T)@A
    Phat,s,QT=np.linalg.svd(B)
    Q=QT.T
    S=np.diag(s)[:k,:k]
    P=V@Phat[:,:k]
    _,cols=S.shape
    return P@S@Q[:,:k].T

In [ ]:
#students are to come up with arguments for the Randomized SVD algorithm
#one might want to use itertools.product to sweep through a large amount of values
#p,q=
np.linalg.norm(Xtrue-RandSVD(Y1,2,p,q),'fro')